# Task Scheduling Project
## Run in Google Colab

This notebook will automatically:
1. Install .NET SDK
2. Clone the repository from GitHub
3. Restore and build the project
4. Run all experiments
5. Display results

**Instructions:**
- Click "Runtime" → "Run all" (or press Ctrl+F9)
- Wait for all cells to complete
- Results will be displayed at the bottom


In [14]:
# Step 1: Install system dependencies and .NET SDK 8.0
print("=" * 70)
print("Step 1: Installing .NET SDK 8.0")
print("=" * 70)

# Install system dependencies
!apt-get update -qq
!apt-get install -y -qq libicu-dev libunwind8 libssl-dev curl

# Download and install .NET SDK 8.0
!wget -q https://dot.net/v1/dotnet-install.sh -O dotnet-install.sh
!bash dotnet-install.sh --channel 8.0 --install-dir /root/.dotnet --verbose

# Set environment variables
import os
os.environ['PATH'] = '/root/.dotnet:/root/.dotnet/tools:' + os.environ.get('PATH', '')
os.environ['DOTNET_ROOT'] = '/root/.dotnet'

# Verify installation
print("\nVerifying .NET installation...")
result = !export PATH=$PATH:/root/.dotnet:/root/.dotnet/tools && dotnet --version 2>&1
if result and len(result) > 0:
    print(f"✅ .NET version: {result[0]}")
else:
    print("❌ .NET installation failed. Trying alternative method...")
    !bash -c 'export PATH=$PATH:/root/.dotnet && dotnet --version'


Step 1: Installing .NET SDK 8.0
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
dotnet-install: Note that the intended use of this script is for Continuous Integration (CI) scenarios, where:
dotnet-install: - The SDK needs to be installed without user interaction and without admin rights.
dotnet-install: - The SDK installation doesn't need to persist across multiple CI runs.
dotnet-install: To set up a development environment or to run apps, use installers rather than this script. Visit https://dotnet.microsoft.com/download to get the installer.

dotnet-install: Calling: machine_has curl
dotnet-install: Calling: calculate_vars 
dotnet-install: Calling: get_normalized_architecture_from_architecture <auto>
dotnet-install: Calling: get_machine_architecture 
dotnet-install: Normalized architecture: 'x64'.
dotnet-install: Calling: get_normalized_os 
do

In [15]:
# Step 2: Clone repository from GitHub
print("=" * 70)
print("Step 2: Cloning Repository from GitHub")
print("=" * 70)

import os
import shutil

# Start from /content directory to avoid nesting issues
if os.path.exists('/content'):
    os.chdir('/content')
    print(f"Changed to /content directory")

# Remove existing directory if it exists
if os.path.exists('Task-Scheduling'):
    print("Removing existing Task-Scheduling directory...")
    shutil.rmtree('Task-Scheduling')

# Clone repository
print("Cloning repository...")
!git clone https://github.com/MohamadJehad/Task-Scheduling.git

# Change to project directory
if os.path.exists('Task-Scheduling'):
    os.chdir('Task-Scheduling')
    print(f"\n✅ Repository cloned successfully!")
    print(f"📁 Current directory: {os.getcwd()}")
    print(f"📁 Project file exists: {os.path.exists('TaskScheduling.csproj')}")
else:
    print("❌ Error: Task-Scheduling directory not found after clone")


Step 2: Cloning Repository from GitHub
Changed to /content directory
Removing existing Task-Scheduling directory...
Cloning repository...
Cloning into 'Task-Scheduling'...
remote: Enumerating objects: 191, done.
remote: Counting objects: 100% (191/191), done.
remote: Compressing objects: 100% (86/86), done.
remote: Total 191 (delta 114), reused 166 (delta 92), pack-reused 0 (from 0)
Receiving objects: 100% (191/191), 251.66 KiB | 1.70 MiB/s, done.
Resolving deltas: 100% (114/114), done.

✅ Repository cloned successfully!
📁 Current directory: /content/Task-Scheduling
📁 Project file exists: True


In [16]:
# Step 3: Restore NuGet packages
print("=" * 70)
print("Step 3: Restoring NuGet Packages")
print("=" * 70)

import os
os.environ['PATH'] = '/root/.dotnet:/root/.dotnet/tools:' + os.environ.get('PATH', '')

# Restore packages
result = !bash -c 'export PATH=$PATH:/root/.dotnet:/root/.dotnet/tools && cd Task-Scheduling && dotnet restore' 2>&1

if any('error' in line.lower() or 'failed' in line.lower() for line in result):
    print("\n❌ Restore failed. Output:")
    for line in result:
        print(line)
else:
    print("\n✅ Packages restored successfully!")


Step 3: Restoring NuGet Packages

✅ Packages restored successfully!


In [17]:
# Step 4: Build the project
print("=" * 70)
print("Step 4: Building Project")
print("=" * 70)

import os
os.environ['PATH'] = '/root/.dotnet:/root/.dotnet/tools:' + os.environ.get('PATH', '')

# Build project
result = !bash -c 'export PATH=$PATH:/root/.dotnet:/root/.dotnet/tools && cd Task-Scheduling && dotnet build --no-restore' 2>&1

if any('error' in line.lower() or 'failed' in line.lower() for line in result):
    print("\n❌ Build failed. Output:")
    for line in result:
        print(line)
else:
    print("\n✅ Project built successfully!")


Step 4: Building Project

✅ Project built successfully!


In [18]:
# Step 5: Run the project
print("=" * 70)
print("Step 5: Running Task Scheduling Experiments")
print("=" * 70)
print("\nThis may take a few minutes...\n")

import os
import subprocess
import sys

os.environ['PATH'] = '/root/.dotnet:/root/.dotnet/tools:' + os.environ.get('PATH', '')
os.environ['DOTNET_ROOT'] = '/root/.dotnet'

# Ensure we're in the project directory (should already be there from Step 2)
current_dir = os.getcwd()
print(f"Current directory: {current_dir}")

# If we're not in Task-Scheduling, navigate to it
if os.path.basename(current_dir) != 'Task-Scheduling':
    if os.path.exists(os.path.join(current_dir, 'Task-Scheduling')):
        os.chdir(os.path.join(current_dir, 'Task-Scheduling'))
        print(f"Changed to: {os.getcwd()}")
    else:
        print("⚠️  Warning: Task-Scheduling directory not found in current location")

project_dir = os.getcwd()
print(f"Running from: {project_dir}\n")

# Verify we have the project file
if not os.path.exists(os.path.join(project_dir, 'TaskScheduling.csproj')):
    print(f"❌ Error: TaskScheduling.csproj not found in {project_dir}")
    print("Available files:")
    for item in os.listdir(project_dir)[:10]:  # Show first 10 items
        print(f"  - {item}")
else:
    # Check if build output exists
    build_output = os.path.join(project_dir, 'bin', 'Debug', 'net8.0')
    executable_exists = os.path.exists(os.path.join(build_output, 'TaskScheduling'))

    print(f"Working directory: {os.getcwd()}")
    print(f"Project directory: {project_dir}")
    print(f"Build output exists: {os.path.exists(build_output)}")
    print(f"Executable exists: {executable_exists}\n")

    # Run the project - use 'dotnet run' which will build if needed
    print("Running project (will build if needed)...\n")

    # Use dotnet run without --no-build to ensure it builds first
    !bash -c 'export PATH=$PATH:/root/.dotnet:/root/.dotnet/tools && export DOTNET_ROOT=/root/.dotnet && cd /content/Task-Scheduling && dotnet run'

    # Check if reports directory was created
    print("\n" + "="*70)
    print("Checking for reports directory...")
    reports_path = os.path.join(project_dir, 'reports')
    if os.path.exists(reports_path):
        report_files = [f for f in os.listdir(reports_path) if f.endswith('.txt')]
        print(f"✅ Reports directory found: {reports_path}")
        print(f"✅ Found {len(report_files)} report file(s)")
        for f in sorted(report_files)[:10]:  # Show first 10
            file_path = os.path.join(reports_path, f)
            size = os.path.getsize(file_path)
            print(f"   📄 {f} ({size:,} bytes)")
        if len(report_files) > 10:
            print(f"   ... and {len(report_files) - 10} more files")
    else:
        print(f"⚠️  Reports directory not found at: {reports_path}")
        print(f"\nCurrent directory contents:")
        try:
            for item in sorted(os.listdir(project_dir)):
                item_path = os.path.join(project_dir, item)
                if os.path.isdir(item_path):
                    print(f"   📂 {item}/")
                    # Check if it's bin or obj
                    if item in ['bin', 'obj']:
                        print(f"      (build artifacts)")
                elif item.endswith(('.csproj', '.sln', '.cs')):
                    print(f"   📄 {item}")
        except Exception as e:
            print(f"   Error listing directory: {e}")


Step 5: Running Task Scheduling Experiments

This may take a few minutes...

Current directory: /content/Task-Scheduling
Running from: /content/Task-Scheduling

Working directory: /content/Task-Scheduling
Project directory: /content/Task-Scheduling
Build output exists: False
Executable exists: False

Running project (will build if needed)...

==═══════════════════════════════════════════════════════════════════════════════
                    MASTER SUMMARY REPORT
         All Algorithms vs All Problem Instances
═══════════════════════════════════════════════════════════════════════════════

Generated: 2025-11-11 22:59:05

═══════════════════════════════════════════════════════════════════════════════
SECTION 1: MAX LOAD COMPARISON
═══════════════════════════════════════════════════════════════════════════════

Instance                  Brute Force  No SortingSort Loads DescSort Loads Desc + T
─────────────────────────────────────────────────────────────────────────
small_0          